# Macrophages_mice （from myeloid)

In [ ]:
#%matplotlib nbagg
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc
import pandas as pd
from os import listdir
from os.path import isfile, join
import re
import anndata
import seaborn as sns

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()

from matplotlib.colors import LinearSegmentedColormap
cmap = LinearSegmentedColormap.from_list(name='gene_cmap', colors=['lightgrey', 'thistle', 'red', 'darkred']) 

sc.settings.set_figure_params(dpi=80, color_map='viridis', vector_friendly=False,  dpi_save=300)

Recluster using all myeloid cells to improve accuracy

# Recalculate UMAP

In [ ]:
adata = sc.read('../../rev_MI/Analysis_mouse/WithCyclings/Outer/Myeloid.h5ad')

In [ ]:
adata.shape

In [ ]:
adata = anndata.AnnData(X=adata.raw.X, obs=adata.obs, var=adata.raw.var)
adata.raw = adata
# Cells look ok, filter out non-expressing genes
sc.pp.filter_genes(adata, min_cells=1)
# find highly variable genes
# parameters are mainly selected depends on user preference
sc.pp.highly_variable_genes(
    adata, min_mean=0.02, max_mean=3, min_disp=0.3, batch_key='StudyID', inplace=True)
sc.pl.highly_variable_genes(adata)
print(np.sum(adata.var.highly_variable))
adata = adata[:, adata.var.highly_variable].copy()
# scale for pca
sc.pp.scale(adata)
# pca
sc.tl.pca(adata, svd_solver='arpack')
sc.external.pp.harmony_integrate(adata, key='Index', adjusted_basis = 'X_pca_harmony_Index_combined')
sc.pp.neighbors(adata, use_rep = 'X_pca_harmony_Index_combined')
sc.tl.umap(adata)
sc.pl.umap(adata, color= ['Condition', 'Day', 'Age', 'StudyID'], wspace=0.3)

In [ ]:
sc.pl.umap(adata, color= ['Condition', 'Day', 'Age', 'StudyID'], wspace=0.3)

In [ ]:
adata.obs.Development_stage.value_counts()

In [ ]:
adata.write('../../rev_MI/Analysis_mouse/WithCyclings/Outer/Myeloid_processed.h5ad')

# Clustering

In [ ]:
adata = sc.read('../../rev_MI/Analysis_mouse/WithCyclings/Outer/Myeloid_processed.h5ad')

In [ ]:
sc.tl.leiden(adata, resolution=0.3, key_added='Leiden_myeloid')

In [ ]:
sc.pl.umap(adata, color='Leiden_myeloid', legend_loc='on data')

In [ ]:
adata.obs.Leiden_myeloid.value_counts()

In [ ]:
sc.pl.umap(adata, color=['Fn1', 'Tcf21', 'Wt1', 'Acta2', 'Pecam1','Aurkb'], color_map=cmap)

In [ ]:
sc.tl.leiden(adata, resolution=1, key_added='Leiden_new')

In [ ]:
adata.write('../../rev_MI/Analysis_mouse/WithCyclings/Outer/Myeloid_processed.h5ad')

# Subset and annotate macrophages

In [ ]:
adata = sc.read('../../rev_MI/Analysis_mouse/WithCyclings/Outer/Myeloid_processed.h5ad')

In [ ]:
GLfinal = {
    'Ma': ['C1qc', 'Apoe', 'Mertk','Cd68', 'Cd63'],
    'Ma-TR': ['Cx3cr1', 'Cd81'],
    'Ma-TR-Cd163': ['Cd163', 'Maf', 'Igf1'],
    'Ma-TR-Cd83+': ['Cd83', 'Il10'],
    'Ma-TR-Cd83-': ['Cd74', 'Ccr2'],
    'Ma-LAM': ['Trem2'],
    'Ma-MD-Gdf15': ['Gdf15','Mpeg1'],
    'Ma-MD-Prdx1': ['Prdx1', 'Lgals3','Anxa1','Ftl1'],
    'Ma-MD-Cxcl3': [ 'Cxcl3','Ccl2', 'Ccl9', 'Arg1'],
    'Ma-MD-Isg15': ['Isg15', 'Irf7', 'Ifit3'],
    'Ma-Fn1': ['Saa3', 'Itgam', 'Fn1'],
    'Ma-Fibro': ['Col1a1', 'Col1a2','Postn', 'Acta2'],
    'Ma-Endo': ['Pecam1', 'Cdh5'],
    'Ma-Cycling': ['Tubb5', 'Stmn1']
}

In [ ]:
ct = {
    '0': 'Neu-1', # S100a8, Cxcr2
    '1': 'Ma-TR-Cd83+', # C1qa, Cd83 -> MHCII+ & MHCII+Il1b+ (MAC-TR)
    '2': 'Mono-1', # Plac8, Ly6e  
    '3': 'Ma-MD-Prdx1',# Cxcl3, Fn1, Ccr2, Arg1 -> Trem2+Spp1+ (M1_Mole) (M1Ma)
    '4': 'Ma-MD-Cxcl3', # C1qa, Lgals3, Prdx1 (https://pubmed.ncbi.nlm.nih.gov/32295421/) -> Trem2+Prdx1+ (M2_Mole) (M2Ma)
    '5': 'Ma-TR-Cd163', # C1qa, Cd163, Siglec1, Hpgd -> TIMD4+ (MAC-TR)
    '6': 'Ma-Cycling',# Stmn1, Tubb5
    '7': 'Ma-lowGC', # C1qa, Rpgrip1, Cd74 (https://pubmed.ncbi.nlm.nih.gov/27464507/) 
    '8': 'DC-1', # Naaa, Cd74
    '9': 'Ma-Fn1', # Saa3, Fn1, Itgam, Ifitm6, Cd68low -> Fn1+Ltc4s+ (MAC8) 
    '10': 'Ma-TR-Cd83-', # C1qa, Hpgd, Cd74 -> MHCII+CCR2+ (MAC-TR)
    '11': 'Ma-MD-Isg15', # IFN1-inducible genes, Isg15, Irf7, IFNIC, Ms4a4c-> Isg15+ (MAC-IFNIC)
    '12': 'Ma-MD-Mpeg1',
    '13': 'Mono-2',# Plac8, Ly6e
    '14': 'Ma-Fibro', # Col1a1
    '15': 'Mono-3', # Plac8, (Ccr2)
    '16': 'Ma-MD-Ccl2',
    '17': 'Ma-Endo',
    '18': 'Neu-3', # Cxcr2, Grk4, Actc1
    '19': 'DC-1', # Naaa, Cd74
    '20': 'Neu-2',# S100a8
    '21': 'NA', 
    '22': 'Ma-Fibro', # (Atca2)
    '23': 'Mono-new', # Plac8
    '24': 'Neu-3', # Cxcr2, Grk4, Actc1
    '25': 'DC-2', # Cd74
    '26': 'Unknown',
}

In [ ]:
adata.obs.Leiden_new.value_counts()

In [ ]:
sc.pl.dotplot(adata, groupby='Leiden_new',var_names=GLfinal, cmap=cmap, swap_axes=True, standard_scale='var'
              #, save="Myeloid_Farhebi.pdf"
             )

In [ ]:
adata.obs['Nctsub3'] = [ct[x] for x in adata.obs.Leiden_new]
adata.obs['Nctsub3'] = adata.obs['Nctsub3'].astype('category')

In [ ]:
adata.uns['Nctsub3_colors']=adata.uns['ctsub3_old_colors']

In [ ]:
adata.obs.ctsub3_old.cat.categories

In [ ]:
adata.obs.Nctsub3.cat.categories

In [ ]:
adata.obs.Nctsub3 = adata.obs.Nctsub3.cat.reorder_categories(['DC-1', 'DC-2', 'Ma-Cycling', 'Ma-Endo', 'Ma-Fibro', 'Ma-Fn1',
       'Ma-MD-Cxcl3', 'Ma-MD-Isg15', 'Ma-MD-Mpeg1', 'Ma-MD-Prdx1',
        'Ma-TR-Cd83+', 'Ma-TR-Cd83-',  'Ma-TR-Cd163','Ma-lowGC', 'Mono-1',
       'Mono-2', 'Mono-3', 'Mono-new', 'NA', 'Neu-1', 'Neu-2', 'Neu-3','Ma-MD-Ccl2',
       'Unknown'])

In [ ]:
sc.pl.umap(adata, color='Nctsub3',legend_loc='on data',legend_fontsize=8)

In [ ]:
adata_ma = adata[adata.obs.Nctsub3.str.startswith('Ma')]

In [ ]:
adata_ma.write('../../rev_MI/WithCyclings/Macro_all.h5ad')

In [ ]:
adata.write('../../rev_MI/WithCyclings/Myeloid_final.processed')

In [ ]:
adata_ma.obs.Nctsub3.value_counts()

In [ ]:
sc.tl.rank_genes_groups(adata, groupby='Nctsub3', n_genes=adata.raw.shape[1], method='t-test', key_added='DE_Nctsub3')

In [ ]:
sc.pl.rank_genes_groups_dotplot(adata, groupby='Nctsub3',key='DE_Nctsub3', cmap=cmap, swap_axes=True, standard_scale='var',n_genes=5,dendrogram=False)

In [ ]:
df = pd.DataFrame(adata.uns['DE_Nctsub3']['names'])

In [ ]:
df.head(100).to_csv('../../rev_MI/WithCyclings/Outer/Mouse100MarkerGenes.csv')

In [ ]:
adata_ma.obs.Nctsub3.cat.categories

In [ ]:
adata_ma.obs.Development_stage.value_counts()

In [ ]:
mac_neo = adata_ma[adata_ma.obs.Development_stage=='neonatal']

In [ ]:
mac_neo.write('./WithCyclings/Macro_neo.h5ad')

# Extract adult mice data only

In [ ]:
Fdata = anndata.AnnData(X=adata_ma.raw.X,obs=adata_ma.obs,obsm=adata_ma.obsm,var=adata_ma.raw.var)

In [ ]:
Fdata = Fdata[Fdata.obs.Development_stage=='adult']

In [ ]:
Fdata.write('../../MI_data/FinalH5ad/Outer/Mouse_ma_final.h5ad')

## Exclude Sham/IR

In [ ]:
Fdata = sc.read('../../MI_data/FinalH5ad/Outer/Mouse_ma_final.h5ad')

In [ ]:
Fdata.obs.Condition.value_counts()

In [ ]:
Fdata = Fdata[np.in1d(Fdata.obs.Condition,['MI','Normal'])]

In [ ]:
Fdata.shape

In [ ]:
Fdata.write('Final_write/mouse_mac.h5ad')